In [2]:
import cv2
import pandas as pd
import os
from collections import defaultdict

# Read the B-SOiD labels
def read_bsoid_labels(csv_path):
    df = pd.read_csv(csv_path)
    return df['B-SOiD labels'].tolist()  # Adjust the column name if different

# Extract frames from video
def extract_frames(video_path, frames_dir):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite(os.path.join(frames_dir, f"frame{count}.jpg"), image)  # save frame as JPEG file      
        success, image = vidcap.read()
        count += 1
    vidcap.release()
    return count  # Return the number of frames

# Group frames by B-SOiD label
def group_frames_by_label(frames_dir, labels):
    frames_groups = defaultdict(list)
    for i, label in enumerate(labels):
        frame_path = os.path.join(frames_dir, f"frame{i}.jpg")
        if os.path.exists(frame_path):
            frames_groups[label].append(frame_path)
    return frames_groups

# Create videos from frames groups
def create_videos_from_groups(frames_groups, output_dir):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 files
    for label, frames in frames_groups.items():
        if frames:  # If there are frames for this label
            video_path = os.path.join(output_dir, f"326_cluster{label}.mp4")
            # Assume that all frames have the same size
            height, width, layers = cv2.imread(frames[0]).shape
            video = cv2.VideoWriter(video_path, fourcc, 30, (width, height))
            for frame_path in frames:
                frame = cv2.imread(frame_path)
                video.write(frame)
            video.release()

# Main function
def main(video_path, csv_path):
    frames_dir = r'D:\deeplabcut\control_pfa\326\frames'  # Directory to save frames
    output_dir = r'D:\deeplabcut\control_pfa\326\output_videos'  # Directory to save output videos
    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    labels = read_bsoid_labels(csv_path)
    num_frames = extract_frames(video_path, frames_dir)
    frames_groups = group_frames_by_label(frames_dir, labels[:num_frames])
    create_videos_from_groups(frames_groups, output_dir)

# Usage
video_path = r'D:\deeplabcut\control_pfa\326.mp4'  # Your video file path
csv_path = r'D:\deeplabcut\control_pfa\BSOID\Oct-27-2023bout_lengths_30Hz326DLC_resnet50_SP_behaviorOct10shuffle1_800000.csv'  # Your CSV file path
main(video_path, csv_path)
